In [120]:
import csv
import sqlparse as sp
from sqlparse.sql import IdentifierList, Identifier
from sqlparse.tokens import Keyword, DML, Newline, Whitespace, Text

In [18]:
'''Data loading functions'''

def extract_metadata():
    path = ""
    metafile = open('files/metadata.txt', 'r') 
    metalines = metafile.readlines() 
    tables_dict = {}
    tables_meta = {}
    tables_list = {}
    is_rec = False
    is_tname = False
    cur_table = ""
    for line in metalines:
        if line.startswith('<begin_table>'):
            is_rec = True
            is_tname = True
        elif line.startswith('<end_table>'):
            if_rec = False
        elif is_tname:
            cur_table = str(line).strip()
            tables_dict[cur_table] = {}
            tables_meta[cur_table] = []
            tables_list[cur_table] = []
            is_tname = False
        else:
            tables_dict[cur_table][str(line).strip()] = []
            tables_meta[cur_table].append(str(line).strip())
    return tables_meta,tables_dict,tables_list
            
def extract_csvdata_bycols(tables_dict):
    tables_data = tables_dict
    for tn,cl in tables_dict.items(): 
        with open('files/'+tn+'.csv', newline='') as table_file:
            all_data = csv.reader(table_file,delimiter=',')
#             print(all_data)
            for row in all_data:
                for c,i in zip(cl,range(len(cl))):
                    tables_data[tn][c].append(int(row[i]))
    return tables_data

def extract_csvdata_byrows(tables_list):
    tables_data = tables_list
    for tn in tables_list: 
        with open('files/'+tn+'.csv', newline='') as table_file:
            all_data = csv.reader(table_file,delimiter=',')
#             print(all_data)
            for row in all_data:
                tables_data[tn].append([int(x) for x in row])
    return tables_data
    

In [20]:
tables_meta,tables_dict,tables_list = extract_metadata()
print(tables_meta)
print(tables_list)
tables_data_bycols = extract_csvdata_bycols(tables_dict)
print(tables_data_bycols)
tables_data_byrows = extract_csvdata_byrows(tables_list)
print(tables_data_byrows)

{'table1': ['A', 'B', 'C'], 'table2': ['D', 'E']}
{'table1': [], 'table2': []}
{'table1': {'A': [922, 640, 775, -551, -952, -354, -497, 411, -900, 858], 'B': [158, 773, 85, 811, 311, 646, 335, 803, 718, 731], 'C': [5727, 5058, 10164, 1534, 1318, 7063, 4549, 10519, 9020, 3668]}, 'table2': {'D': [158, 773, 85, 811, 311, 646, 335, 803, 718, 731], 'E': [11191, 14421, 5117, 13393, 16116, 5403, 6309, 12262, 10226, 13021]}}
{'table1': [[922, 158, 5727], [640, 773, 5058], [775, 85, 10164], [-551, 811, 1534], [-952, 311, 1318], [-354, 646, 7063], [-497, 335, 4549], [411, 803, 10519], [-900, 718, 9020], [858, 731, 3668]], 'table2': [[158, 11191], [773, 14421], [85, 5117], [811, 13393], [311, 16116], [646, 5403], [335, 6309], [803, 12262], [718, 10226], [731, 13021]]}


In [178]:
'''SQL functions'''

def remove_wspaces(parsed_sql):
    modf_parsed_sql = []
    for token in parsed_sql:
        if token.is_whitespace:
            continue
        modf_parsed_sql.append(token)
    return modf_parsed_sql

# def fn_select(qry_token_list):


def process_where(where_stmnt):
    where_stmnt = sp.parse(where_stmnt)[0]
    modf_where_stmnt = remove_wspaces(where_stmnt)
    return []
#     curr_token = ""
#     cndt_and,cndt_or = [],[]
#     for token in 
    
    

def process_query(parsed_sql):
    modf_parsed_sql = remove_wspaces(parsed_sql)
    print(modf_parsed_sql)
    curr_token = ""
    q_columns,q_tables,q_conditions = [],[],[]
    for token in modf_parsed_sql:
#         print('====================',token.value,token.ttype)
        if token.ttype is DML and token.value == 'select':
            curr_token = 'select'
            continue
        if token.ttype is Keyword and token.value == 'from':
#             print('fffffffffffff')
            curr_token = 'from'
            continue
        if curr_token == 'select':
            q_columns = token
        elif curr_token == 'from':
#             print("++++++++",token)
            q_tables = token
            curr_token = 'where'
        elif curr_token == 'where':
            q_conditions = []
#     print("%%%%%%%%%%%%",str(modf_parsed_sql[-1][0])=='where')
    return q_columns,q_tables,q_conditions
#         if token.ttype is Text.Whitespace.Newline:
#             print("*************")
#         print(token.ttype)
#         if token in ['select','from','where','and','or']
    
#     for qline in parsed_sql.split('\n'):
#         modf_parsed_sql.append(remove_wspaces(sp.parse(qline)))
#     qry_outpt = []
#     if modf_parsed_sql[0].lower() == 'select':
#         qry_output = fn_select(modf_parsed_sql)
#     return modf_parsed_sql
        

In [179]:
# qry_input = input().strip().lower()
qry_input = "select * from t1, t2 where c1=1 and c2=2".lower()
frmt_qry = sp.format(qry_input,reindent=True, keyword_case='upper')
# print(frmt_qry[1])
parsed_sql = sp.parse(qry_input)[0]
# print(parsed_sql.tokens)
q_columns,q_tables,q_conditions = process_query(parsed_sql)
print(q_columns)
print(q_tables)
print(q_conditions)

[<DML 'select' at 0x7F82841C0B28>, <Wildcard '*' at 0x7F82840B38E8>, <Keyword 'from' at 0x7F82840B3408>, <IdentifierList 't1, t2' at 0x7F82840B0480>, <Where 'where ...' at 0x7F8284121480>]
*
t1, t2
[]


In [41]:
# qry_input = input().strip()
# parsed_sql = sp.parse(qry_input)[0]
# print(parsed_sql)
# print(parsed_sql.tokens)
# print(sp.split(qry_input))
# print(qry_output[0])
# print(qry_output[1][0])
print(qry_output[4][2])

c=0


In [30]:
print(parsed_sql.tokens[1])
print(parsed_sql.flatten())
for token in parsed_sql.tokens:
    if(token.is_whitespace):
        print("w")
    print(token)

 
<generator object TokenList.flatten at 0x7f82841600f8>
select
w
 
a, b
w
 
from
w
 
tab1,tab2
